In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# SWIFT PAT+OBS: Calculate MMC performance (Table 2)

## I/O data

In [2]:
simulations = {
    'case A0': 'internal.ipa.obs.noT',
    'case A1': 'internal.ipa.obs.Tassim',
    'case An': 'internal.ipa.obs.allT',
    'case B0': 'internal.dpa.obs.noT',
    'case B1': 'internal.dpa.obs.Tassim',
    'case Bn': 'internal.dpa.obs.allT',
    'case Ca': 'internal.ipa.obs.wrfT',
    'case Cb': 'internal.ipa.obs.wrfTadv',
    'case R' : 'internal.ipa.wrf',  
}

In [3]:
datadir = '/scratch/dallaert/profile_assimilation_obs'

## Some definitions

In [4]:
# Dimensions of reference rotor span
hubheight = 90       # Hub height of interest
diameter  = 120   # Rotor diameter of interest

In [5]:
# Gather statistics for selected fields
selected_fields = ['wspd','wdir','thetav','thetav_gradient','alpha','psi','TKE','TI']

# Metrics that will be considered
selected_metrics = ['MAE','ME','STD','NMAE','MEAN']

# Gather statistics for the entire day
interval = '23h50min'

## Prepare environment

In [6]:
import os
import pandas as pd
import xarray as xr

In [7]:
from helper_functions import calc_QOIs,calc_rotor_average

## Load simulation data

Load 10-min data

In [8]:
%%time
df_10min = {}
for name,sim in simulations.items():
    print('Processing '+name)
    # Read 10min data
    df_10min[name] = xr.open_dataset(os.path.join(datadir,sim,'probe1_10min.nc')).to_dataframe()
    # Calculate some quantities of interest
    calc_QOIs(df_10min[name])

Processing case A0
Processing case A1
Processing case An
Processing case B0
Processing case B1
Processing case Bn
Processing case Ca
Processing case Cb
Processing case R
CPU times: user 5.96 s, sys: 127 ms, total: 6.08 s
Wall time: 6.31 s


Calculate rotor-averaged quantities

In [10]:
df_ravg = {}
for sim in simulations:
    df_ravg[sim] = calc_rotor_average(df_10min[sim],hubheight,diameter)

## Load reference data

Reference data:
- **WRF mesoscale:** NetCDF file generated in `wrf_to_sowfa.ipynb`
- **TTU tower (10-min statistics):** Dataset generated in `process_TTU_tower.ipynb` which can be found in the [a2e-mmc/assessment repository](https://github.com/a2e-mmc/assessment)

In [11]:
from helper_functions import load_TTU_tower_reference_data

Load reference data

In [12]:
tower_10min = load_TTU_tower_reference_data('../TTU_tilt_corrected_20131108-09_10min.csv')

Calculate rotor-averaged quantities

In [13]:
df_ravg['tower'] = calc_rotor_average(tower_10min,hubheight,diameter)

# Integral measures

In [14]:
selected_times = pd.date_range('2013-11-08 12:00:00', '2013-11-09 11:49:00',freq=interval)

In [15]:
# Create index
midx = pd.MultiIndex.from_product([selected_times, # Selected times
                                   selected_metrics,                        # Metrics
                                   df_10min.keys()],                 # List of simulations including wrf
                                  names=['datetime','metric','sim'])

# Create field of NaNs to fill dataframe
nanfield = np.nan*np.ones((midx.size,len(selected_fields)))

# Create dataframe
metrics = pd.DataFrame(nanfield,columns=selected_fields,index=midx)

In [16]:
# Loop over different time periods
for tstart in selected_times:
    # Start and end of the period for which error metrics are calculated
    tend   = tstart + pd.to_timedelta(interval)
    
    # Get observed rotor-averaged quantities for current period
    obs = df_ravg['tower']
    obs = obs.loc[(obs.index.get_level_values(0)>=tstart) &\
                   (obs.index.get_level_values(0)<=tend)].copy()
    
    # Loop over different datasets
    for dfname in df_10min.keys():
            
        # Get simulated rotor-averaged quantities for current period
        df  = df_ravg[dfname]
        df  = df.loc[(df.index.get_level_values(0)>=tstart) &\
                                   (df.index.get_level_values(0)<=tend)].copy()
        
        # Loop over different quantities of interest
        for field in selected_fields:
            try:
                # Compute metrics and store in dataframe
                metrics.loc[(tstart,'MAE',dfname),field] = np.nanmean(np.abs(df[field].values-obs[field].values))
                metrics.loc[(tstart,'NMAE',dfname),field] = np.nanmean(np.abs(df[field].values-obs[field].values))#/np.mean(obs[field].values)
                metrics.loc[(tstart,'ME',dfname),field]  = (df[field].values-obs[field].values).mean()
                metrics.loc[(tstart,'STD',dfname),field] = (df[field].values-obs[field].values).std()
                metrics.loc[(tstart,'MEAN',dfname),field] = np.mean(df[field].values)
            except KeyError:
                pass

In [17]:
metrics.loc[(slice(None),'NMAE',slice(None)),:] = metrics.loc[(slice(None),'NMAE',slice(None)),:]/metrics.xs('MAE',level='metric').xs('case R',level='sim')

In [18]:
metrics.xs('2013-11-08 12:00:00',level='datetime').loc[(('MAE','NMAE'),slice(None)),:].unstack(level=0)

wspd                wdir              thetav            \
metric        MAE      NMAE       MAE      NMAE       MAE      NMAE   
sim                                                                   
case A0  0.883288  0.430925  4.408923  0.618943  2.197701  1.210752   
case A1  0.847444  0.413438  3.637470  0.510643  0.187753  0.103436   
case An  0.911127  0.444507  4.175735  0.586207  0.725997  0.399964   
case B0  0.921151  0.449397  3.799755  0.533425  3.276718  1.805201   
case B1  1.143158  0.557706  4.741517  0.665634  0.142493  0.078502   
case Bn  0.533680  0.260364  2.114531  0.296847  0.798528  0.439923   
case Ca  0.747630  0.364742  3.755228  0.527174  1.644795  0.906146   
case Cb  0.931511  0.454451  4.271722  0.599682  1.811479  0.997975   
case R   2.049749  1.000000  7.123313  1.000000  1.815154  1.000000   

        thetav_gradient               alpha                 psi            \
metric              MAE      NMAE       MAE      NMAE       MAE      NMAE   
sim                                                                         
case A0       17.245197  1.554157  0.144031  1.185629  0.130734  2.047477   
case A1        6.963306  0.627541  0.098500  0.810828  0.076123  1.192195   
case An        7.625325  0.687203  0.113002  0.930210  0.086915  1.361218   
case B0       10.639901  0.958880  0.081845  0.673729  0.054248  0.849603   
case B1       10.768585  0.970477  0.084101  0.692302  0.068601  1.074394   
case Bn        1.749315  0.157650  0.067299  0.553987  0.039150  0.613145   
case Ca       13.897430  1.252452  0.133928  1.102462  0.110795  1.735217   
case Cb       12.023223  1.083547  0.134703  1.108847  0.104677  1.639394   
case R        11.096176  1.000000  0.121480  1.000000  0.063851  1.000000   

               TKE                   TI            
metric         MAE       NMAE       MAE      NMAE  
sim                                                
case A0   0.549631   0.829234  0.038007  0.821444  
case A1   0.516098   0.778642  0.032908  0.711246  
case An   0.533658   0.805136  0.035336  0.763712  
case B0   9.632690  14.532936  0.228975  4.948854  
case B1  11.278208  17.015544  0.250851  5.421647  
case Bn   2.302987   3.474540  0.084090  1.817431  
case Ca   0.550701   0.830849  0.037754  0.815988  
case Cb   0.527257   0.795477  0.029096  0.628860  
case R    0.662818   1.000000  0.046268  1.000000